In [2]:
import os
import mido
from mido import MidiFile, Message
from threading import Event
from queue import Queue
import simpleaudio as sa

In [3]:
class SimplePlayer:
    do_tick = True

    def __init__(
        self,
        kill_event: Event,
        playback_event: Event,
        filename_queue: Queue,
    ) -> None:
        self.kill_event = kill_event
        self.get_next = playback_event
        self.file_queue = filename_queue

    def playback_loop(self, seed_file_path: str):
        self.playing_file_path = seed_file_path
        self.next_file_path = self.file_queue.get()
        next_file = os.path.basename(self.next_file_path)

        while self.next_file_path is not None and not self.kill_event.is_set():
            self.playing_file = os.path.basename(self.playing_file_path)
            file_tempo = int(os.path.basename(self.playing_file).split("-")[1])
            found_tempo = -1

            for track in MidiFile(self.playing_file_path).tracks:
                for msg in track:
                    if msg.type == "set_tempo":
                        found_tempo = msg.tempo

            print(
                f"playing '{self.playing_file}' ({file_tempo}) --> '{next_file}' ({round(mido.tempo2bpm(found_tempo)):01d})"
            )

            self.get_next.set()
            self.play_midi_file(self.playing_file_path)
            self.playing_file_path = self.next_file_path

        print(f"shutting down")

    def play_midi_file(self, midi_path: str) -> None:
        midi = MidiFile(midi_path)

        with mido.open_output("Disklavier") as outport:  # type: ignore
            for msg in midi.play(meta_messages=True):
                if not msg.is_meta:
                    outport.send(msg)
                if msg.type == "text":  # type: ignore
                    self.tick()

                # check if we're dead yet
                if self.kill_event.is_set():
                    self.end_notes()
                    break

    def end_notes(self):
        with mido.open_output("Disklavier") as outport:  # type: ignore
            for note in range(128):
                msg = Message("note_off", note=note, velocity=0, channel=0)
                outport.send(msg)

    def tick(self):
        tick = sa.WaveObject.from_wave_file("../data/tick.wav")
        tick.play()

In [ ]:
give_p1 = Event()
kill_p1 = Event()
plst_p1 = Queue()

player1 = SimplePlayer()